<a href="https://colab.research.google.com/github/etuckerman/LLaMA-3.1-QA-Finetune/blob/main/LLaMA_3_1_QA_Finetune.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Create examplary dataset (16 qna's)

In [1]:
import pandas as pd

# Create a DataFrame with 16 example questions and answers
data = {
    'Question': [
        "What is the capital of France?",
        "Who wrote 'To Kill a Mockingbird'?",
        "What is the boiling point of water?",
        "Who is known as the father of modern physics?",
        "Which planet is known as the Red Planet?",
        "What is the largest mammal in the world?",
        "Who painted the Mona Lisa?",
        "What is the speed of light?",
        "Who developed the theory of relativity?",
        "What is the hardest natural substance on Earth?",
        "Which element has the chemical symbol 'O'?",
        "What is the smallest unit of life?",
        "Who was the first person to walk on the Moon?",
        "What is the main ingredient in guacamole?",
        "Which country is known as the Land of the Rising Sun?",
        "What is the largest ocean on Earth?"
    ],
    'Answer': [
        "The capital of France is Paris.",
        "Harper Lee wrote 'To Kill a Mockingbird'.",
        "The boiling point of water is 100 degrees Celsius.",
        "Albert Einstein is known as the father of modern physics.",
        "Mars is known as the Red Planet.",
        "The blue whale is the largest mammal in the world.",
        "Leonardo da Vinci painted the Mona Lisa.",
        "The speed of light is approximately 299,792 kilometers per second.",
        "Albert Einstein developed the theory of relativity.",
        "Diamond is the hardest natural substance on Earth.",
        "Oxygen has the chemical symbol 'O'.",
        "The cell is the smallest unit of life.",
        "Neil Armstrong was the first person to walk on the Moon.",
        "The main ingredient in guacamole is avocado.",
        "Japan is known as the Land of the Rising Sun.",
        "The Pacific Ocean is the largest ocean on Earth."
    ]
}

df = pd.DataFrame(data)
df.to_csv('qa_examples.csv', index=False)
